- 直接就这视频往下看是看不懂的，我们有必要先进行一些准备工作

## [词嵌入（Word2vec）](https://zhuanlan.zhihu.com/p/27234078)

- [注解版,结合着看](/%E9%99%84%E5%BD%9518%20%E7%90%86%E8%A7%A3%20Word2Vec%20%E4%B9%8B%20Skip-Gram%20%E6%A8%A1%E5%9E%8B%20-%20%E7%9F%A5%E4%B9%8E.html)

- 什么是词嵌入？
  - Word2Vec是从大量文本语料中以无监督的方式学习语义知识的一种模型，它被大量地用在自然语言处理（NLP）中。
  - 自然语言是用来表达人脑思维的复杂系统。 在这个系统中，词是意义的基本单元。
  - 词向量是用于表示单词意义的向量， 并且还可以被认为是单词的特征向量或表示。 将单词映射到实向量的技术称为词嵌入。
  - 那么它是如何帮助我们做自然语言处理呢？
    - Word2Vec其实就是通过学习文本来用词向量的方式表征词的语义信息，
    - 即通过一个嵌入空间使得语义上相似的单词在该空间内距离很近。
    - Embedding其实就是一个映射，将单词从原先所属的空间映射到新的多维空间中，也就是把原先词所在空间嵌入到一个新的空间中去。

- Word2Vec模型实际上分为了两个部分，第一部分为建立模型，第二部分是通过模型获取嵌入词向量。
- Word2Vec的整个建模过程实际上与自编码器（auto-encoder）的思想很相似，即先基于训练数据构建一个神经网络，
- 当这个模型训练好以后，我们并不会用这个训练好的模型处理新的任务，我们真正需要的是这个模型通过训练数据所学得的参数，
  - 例如隐层的权重矩阵——后面我们将会看到这些权重在Word2Vec中实际上就是我们试图去学习的“word vectors”。基于训练数据建模的过程，
  - 我们给它一个名字叫“Fake Task”，意味着建模并不是我们最终的目的。

### 为何独热向量是一个糟糕的选择

- 我们曾经使用独热向量来表示词（字符就是单词）。假设词典中不同词的数量（词典大小）为，每个词对应一个从到的不同整数（索引）。
- 为了得到索引为的任意词的独热向量表示，我们创建了一个全为0的长度为N的向量，并将位置i的元素设置为1。这样，每个词都被表示为一个长度为N的向量，可以直接由神经网络使用。
- 虽然独热向量很容易构建，但它们通常不是一个好的选择。一个主要原因是独热向量不能准确表达不同词之间的相似度，
- 比如我们经常使用的“余弦相似度”。对于向量，它们的余弦相似度是它们之间角度的余弦：
- $$\frac{\mathbf{x}^\top \mathbf{y}}{\|\mathbf{x}\| \|\mathbf{y}\|} \in [-1, 1].$$
- 独热向量不能反映向量之间的相似性

### 自监督的word2vec
- word2vec工具是为了解决上述问题而提出的。
  - 它将每个词映射到一个固定长度的向量，这些向量能更好地表达不同词之间的相似性和类比关系。
- word2vec工具包含两个模型，
  - 即跳元模型（skip-gram） [Mikolov et al., 2013b](/%E9%99%84%E5%BD%9500%20%E5%8F%82%E8%80%83%E6%96%87%E7%8C%AE/NIPS-2013-distributed-representations-of-words-and-phrases-and-their-compositionality-Paper.pdf)和连续词袋（CBOW） [Mikolov et al., 2013a](/%E9%99%84%E5%BD%9500%20%E5%8F%82%E8%80%83%E6%96%87%E7%8C%AE/Efficient%20Estimation%20of%20Word%20Representations%20in%20Vector%20Space.pdf)。
  - 对于在语义上有意义的表示，它们的训练依赖于条件概率，条件概率可以被看作是使用语料库中一些词来预测另一些单词。由于是不带标签的数据，因此跳元模型和连续词袋都是自监督模型。

这里李沐老师的教材讲的有点乱了，看b乎解释吧

1. The Fake Task
- 上面提到，训练模型的真正目的是获得模型基于训练数据学得的隐层权重。为了得到这些权重，首先要构建一个完整的神经网络作为“Fake Task”，后面再返回来看通过“Fake Task”如何间接地得到这些词向量。

<img src="https://pic4.zhimg.com/80/v2-ca21f9b1923e201c4349030a86f6dc1f_720w.png">

- 一图胜千言，具体解释见[附录18](/%E9%99%84%E5%BD%9518%20%E7%90%86%E8%A7%A3%20Word2Vec%20%E4%B9%8B%20Skip-Gram%20%E6%A8%A1%E5%9E%8B%20-%20%E7%9F%A5%E4%B9%8E.html)
- 两个关键参数：
  - skip_window：这个相当于我们用滑动窗口取词，这个窗口的长度span的左右边界最大长度
  - num_skips: 此参数相当于告诉我们，以中心词构建最多有2倍skip_window个词对，我们选取其中num_skips个作为训练样本
- 这里单词的输入依旧采用了one_hot编码，

### hidden_layer
- 如果我们现在想用300个特征来表示一个单词（即每个词可以被表示为300维的向量）。那么隐层的权重矩阵应该为10000行(one_hot编码)，300列（隐层有300个结点）。
    - Google在最新发布的基于Google news数据集训练的模型中使用的就是300个特征的词向量。词向量的维度是一个可以调节的超参数（在Python的gensim包中封装的Word2Vec接口默认的词向量大小为100， window_size为5）。

- 看下面的图片，左右两张图分别从不同角度代表了输入层-隐层的权重矩阵。左图中每一列代表一个10000维的词向量和隐层单个神经元连接的权重向量。从右边的图来看，每一行实际上代表了每个单词的词向量。
- <img src="picture/v2-c538566f7d627ce7ca40589f15ca8284_720w.png">

所以我们最终的目标就是学习这个隐层的权重矩阵。
我们现在回来接着通过模型的定义来训练我们的这个模型。

额外说一句，独热编码后的矩阵是非常稀疏的，矩阵的计算的结果实际上是矩阵对应的向量中值为1的索引，实际计算中是采用这种方式